- S metaheuristike
    - lokalna pretraga
    - simulirano kaljenje
    - promenljive okoline
    - tabu pretraga
    - iterativna lokalna
    - ...
- P metaheuristike

In [4]:
# uflp - prost lokacijski problem
# I korisnika
# J resursa
# c_ij - cena pridruzivanja korisnika i resursu j
# f_j - cena uspostavljanja resursa j

# J = 5
# resenje: odabir resursa - solution = [True, False, True, False, True]
#                                      [True, False, True, True, True]



# 5 aviona
# solution =    [3,1,2,5,4]
# newSolution = [3,2,1,5,4]

In [6]:
# min f
# max f = min (-f)

In [7]:
import random

In [9]:
def isFeasible(solution):
    for resource in solution:
        if resource:
            return True
    return False

In [8]:
def initialize(numResources, p):
    solution = []
    for i in range(numResources):
        solution.append(random.random() < p)
    if not isFeasible(solution):
        solution[random.randrange(numResources)] = True
    
    return solution

In [10]:
def calcSolutionValue(solution, cost, fixedCost):
    numUsers = len(cost)
    numResources = len(fixedCost)
    value = 0.0
    used = [False for _ in range(numResources)]
    
    for i in range(numUsers):
        minCost = float('inf')
        resourceUsed = -1
        for j in range(numResources):
            if solution[j] and cost[i][j] < minCost:
                minCost = cost[i][j]
                resourceUsed = j
                
        value += minCost
        used[resourceUsed] = True
        
    for j in range(numResources):
        if used[j]:
            value += fixedCost[j]
            
    solution = used
    return value

In [13]:
def invert(solution):
    chosen = random.randrange(len(solution))
    solution[chosen] = not solution[chosen]
    if isFeasible(solution):
        return chosen
    solution[chosen] = not solution[chosen]
    return -1

In [14]:
def restore(solution, j):
    solution[j] = not solution[j]

In [20]:
def localSearch(cost, fixedCost, iters):
    solution = initialize(len(fixedCost), p=0.25)
    currValue = calcSolutionValue(solution, cost, fixedCost)
    bestValue = currValue
    
    for i in range(iters):
#         newSolution = izmeni trenutno resenje - u okolini prethodnog resenja
        j = invert(solution)
        if j < 0:
            continue
        newValue = calcSolutionValue(solution, cost, fixedCost)
        
        if newValue < currValue:
            currValue = newValue
            if newValue < bestValue:
                bestValue = newValue
        else:
#             nastavi petlju sa starim solution
            restore(solution, j)
            
    return bestValue

In [17]:
def readInput(filename):
    with open(filename, 'r') as f:
        numUsers, numResources = [int(x) for x in f.readline().split()]
        cost = [[int(j) for j in f.readline().split()] for i in range(numUsers)]
        fixedCost = [int(x) for x in f.readline().split()]
        
        return numUsers, numResources, cost, fixedCost

In [19]:
numUsers, numResources, cost, fixedCost = readInput('uflp1.txt')
localSearch(cost, fixedCost, iters=10000)

34.0

In [21]:
def simulatedAnnealing(cost, fixedCost, iters):
    solution = initialize(len(fixedCost), p = 0.25)
    currValue = calcSolutionValue(solution, cost, fixedCost)
    bestValue = currValue
    
    for i in range(iters):
        j = invert(solution)
        if j < 0:
            continue
        
        newValue = calcSolutionValue(solution, cost, fixedCost)
        
        if newValue < currValue:
            currValue = newValue
            if newValue < bestValue:
                bestValue = newValue
        else:
            p = 1.0 / i ** 0.5
            q = random.uniform(0, 1)
            if p > q:
                currValue = newValue
            else:
                restore(solution, j)
                
    return bestValue

In [22]:
simulatedAnnealing(cost, fixedCost, iters=10000)

34.0

In [30]:
def getNeighbor(k, solution):
    numResources = len(solution)
    inverted = []
    
    for i in range(k):
        j = random.randrange(numResources)
        while j in inverted:
            j = random.randrange(numResources)
        solution[j] = not solution[j]
        inverted.append(j)
        
    if isFeasible(solution):
        return True, inverted
    else:
        restoreMultiple(solution, inverted)
        return False, inverted

In [31]:
def restoreMultiple(solution, inverted):
    for j in inverted:
        solution[j] = not solution[j]

In [32]:
def RVNS(cost, fixedCost, iters, kMax):
    solution = initialize(numResources=len(fixedCost), p=0.25)
    currValue = calcSolutionValue(solution, cost, fixedCost)
    bestValue = currValue
    
    for i in range(iters):
        k = 0
        while k <= kMax:
            feasible, inverted = getNeighbor(k, solution)
            if not feasible:
                continue
            
            newValue = calcSolutionValue(solution, cost, fixedCost)
            if newValue < currValue:
                currValue = newValue
                k = 0
                if newValue < bestValue:
                    bestValue = newValue
            else:
                restoreMultiple(solution, inverted)
                k += 1
                
    return bestValue

In [33]:
RVNS(cost, fixedCost, iters=10, kMax=3)

34.0